<a href="https://colab.research.google.com/github/ByteBelle-21/maze_solver/blob/main/maze_solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import matplotlib.pyplot as plt
import numpy as np
from google.colab import files
import cv2

In [ ]:
is_valid = False
filename = ""
while not is_valid :
  input_image = files.upload()
  filename = next(iter(input_image))
  is_valid = cv2.haveImageWriter(filename)
  if not is_valid :
    print("File format not supported. Upload new file!")
  else :
    opened_file = open(filename, "wb")
    opened_file.write(input_image[filename])

valid_image = cv2.imread(filename)
if valid_image is None:
  print("File doesn't exist")
  exit(0)
plt.imshow(valid_image)
plt.axis("off")
plt.title("Original Uploaded Image")
plt.show()

In [ ]:
gray_image = cv2.cvtColor(valid_image, cv2.COLOR_BGR2GRAY)
binary_image = cv2.adaptiveThreshold(gray_image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 15, 2)
plt.imshow(binary_image, cmap='gray')
plt.axis("off")
plt.title("Updated Binary Image")
plt.show()

In [ ]:
image_matrix = (binary_image == 255).astype(int)
print(image_matrix)